## Notebook Preface

## Constructs the working folder

* Positions the project folder in the Google Drive.
  1. From "Share with me", right click on "W266 Final Project", select "Add shortcut to Drive"
  2. "W266 Final Project" will show up in "MyDrive"

* Mounts the Google Drive at /content/drive in the Colab runtime.

* Defines the working folder relative to /content/drive.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
working_folder = "/content/drive/MyDrive/W266 Final Project/CnF/PhotoStoryGenerator"
testing_json = f"{working_folder}/test_hints.json"
checkpoint_dir = f"{working_folder}/GPT-2M-FineTune2_checkpoint"
results_json = f"{working_folder}/beamsearch_results.json"
resume = True


In [ ]:
!nvidia-smi


Sat Jul 24 05:28:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    41W / 250W |   9969MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Imports libraries

In [ ]:
# First upload the training and evaluation files to this runtime (Press connect if needed)
!pip install transformers torch


In [ ]:
import json
import logging
import math
import os
import re
from dataclasses import dataclass, field
from typing import Optional

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from transformers import (
    AutoConfig,
    GPT2LMHeadModel,
    AutoTokenizer,
    TextGenerationPipeline,
    pipeline,
    set_seed,
)

# Setup logging
logger = logging.getLogger(__name__)

from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
#METEOR
!pip install nltk==3.5
import nltk
nltk.download('wordnet')

#BLEU
from nltk.translate.bleu_score import sentence_bleu

#ROUGE
!pip install rouge-score
from rouge_score import rouge_scorer
Rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

#BERT SCORE
!pip install bert_score
from bert_score import BERTScorer
BERT_scorer = BERTScorer(lang="en", rescale_with_baseline=True)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Defines score computer

In [ ]:
def get_scores(reference, hypothesis):
    rouge = Rouge_scorer.score(reference, hypothesis)
    P, R, F1 = BERT_scorer.score([reference], [hypothesis])
    return {
        "meteor": nltk.translate.meteor_score.meteor_score([reference], hypothesis),
        "rouge1": { "precision": rouge["rouge1"].precision, "recall": rouge["rouge1"].recall, "fmeasure": rouge["rouge1"].fmeasure },
        "rougeL": { "precision": rouge["rougeL"].precision, "recall": rouge["rougeL"].recall, "fmeasure": rouge["rougeL"].fmeasure },
        "bleu": sentence_bleu(hypothesis, reference, weights=(1, 0, 0, 0)),
        "bert_score": { "precision": P.item(), "recall": R.item(), "f1": F1.item() }
    }


## Constructs the testing dataset

In [ ]:
def load_dataset_for_testing(resume):
    if resume:
        with open (results_json) as f:
            _json = json.load(f)
            return _json
    else:
        results = []
        with open (testing_json) as f:
            _json = json.load(f)
            for s in _json:
                story = _json[s]
                hints = story["hints"]
                results.append({
                    "story_id": s,
                    "reference": story["sis"],
                    "base_prefix": " ".join(hints[1]),
                    "prefix": f"<BOS> <HINT> {' '.join(hints[0])} <SENT>",
                    "urls": story["urls"]
                })
        return results


In [ ]:
data = load_dataset_for_testing(resume)


## Creates the story generation pipeline

In [ ]:
model = GPT2LMHeadModel.from_pretrained(checkpoint_dir)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
story_generator = TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)


In [ ]:
def dump_data(_data):
    with open(f"{results_json}", "w") as outfile:
        json.dump(_data, outfile)
    print("Finished dumping data")
    print("# of processed records:", len([d for d in _data if "beam_search" in d]))


## Text generation with beam search

In [ ]:
story_regex = re.compile("^<BOS> <HINT> [^<>]+ <SENT> (?P<SENT>[^.!?]+[.!?][^.!?]+[.!?][^.!?]+[.!?][^.!?]+[.!?][^.!?]+[.!?]).+", re.MULTILINE|re.IGNORECASE)

num_tests = len(data)

print("# of records:", num_tests)
print("# of processed records:", len([d for d in data if "beam_search" in d]))

num_dumps = 50
set_seed(0)
for d in data:
    if "beam_search" in d:
        continue
    stories = story_generator(
        d["prefix"],
        max_length=500,
        num_beams=5,
        no_repeat_ngram_size=2,
        num_return_sequences=5, 
        early_stopping=False
    )
    best = {"text": "", "scores": {
        "meteor": -100,
        "rouge1": { "precision": -100, "recall": -100, "fmeasure": -100 },
        "rougeL": { "precision": -100, "recall": -100, "fmeasure": -100 },
        "bleu": -100,
        "bert_score": { "precision": -100, "recall": -100, "f1": -100 },
    }}
    for story in stories:
        gen_text = story["generated_text"].replace("\n", "")
        match = story_regex.match(gen_text)
        if match is not None:
            hypothesis = match.group("SENT")
        else:
            hypothesis = gen_text[:len(d["prefix"])]
        if len(hypothesis) == 0:
            continue
        scores = get_scores(d["reference"], hypothesis)
        if scores["meteor"] > best["scores"]["meteor"]:
            best = {"text": hypothesis, "scores": scores}
    d["beam_search"] = best

    num_tests -= 1

    if num_tests % num_dumps == 0:
        dump_data(data)

    if num_tests == 0:
        break

dump_data(data)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# of records: 2764
# of processed records: 164


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

Finished dumping data
# of processed records: 164


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Finished dumping data
# of processed records: 2664


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Finished dumping data
# of processed records: 2714


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Finished dumping data
# of processed records: 2764
Finished dumping data
# of processed records: 2764
